# Analysis locations to determine the coffee shops opening

## 1. Import data

In [1]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_ab68ce2be73a49f4ab5dafebba69a3f7 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='8ICSFFI-cSzrLVDA3K9hl9HAquk8ipNQLplXRMMl5iV9',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_ab68ce2be73a49f4ab5dafebba69a3f7.get_object(Bucket='findlocationstoopencoffeeshopsin3-donotdelete-pr-t03u5r22fef2jv',Key='MajorCities.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

vn = pd.read_csv(body)
vn.head()


,District,City,Area,Population,Latitude/Longitude
0,Ba Dinh,Ha Noi,9.224,247100,"21.036667, 105.836111"
1,Bac Tu Liem,Ha Noi,43.350,333300,"21.070351, 105.762098"
2,Cau Giay,Ha Noi,12.040,266800,"21.028307, 105.790783"
3,Dong Da,Ha Noi,9.960,420900,"21.013825, 105.828677"
4,Hai Ba Trung,Ha Noi,10.090,318000,"21.011667, 105.847778"


In [2]:
#split data
new = vn['Latitude/Longitude'].str.split(",", n = 1, expand = True)
vn['Latitude']= new[0]
vn['Longitude']= new[1]
vn.drop(columns = ['Latitude/Longitude'], inplace = True)
vn.head()

,District,City,Area,Population,Latitude,Longitude
0,Ba Dinh,Ha Noi,9.224,247100,21.036667,105.836111
1,Bac Tu Liem,Ha Noi,43.350,333300,21.070351,105.762098
2,Cau Giay,Ha Noi,12.040,266800,21.028307,105.790783
3,Dong Da,Ha Noi,9.960,420900,21.013825,105.828677
4,Hai Ba Trung,Ha Noi,10.090,318000,21.011667,105.847778


In [3]:
#Cast column to number
vn['Area'] = pd.to_numeric(vn['Area'])
vn['Population'] = pd.to_numeric(vn['Population'])
#Calculate population density
vn['Density'] = vn['Population'].div(vn['Area'], axis = 0)
vn.head()

,District,City,Area,Population,Latitude,Longitude,Density
0,Ba Dinh,Ha Noi,9.224,247100,21.036667,105.836111,26788.811795
1,Bac Tu Liem,Ha Noi,43.350,333300,21.070351,105.762098,7688.581315
2,Cau Giay,Ha Noi,12.040,266800,21.028307,105.790783,22159.468439
3,Dong Da,Ha Noi,9.960,420900,21.013825,105.828677,42259.036145
4,Hai Ba Trung,Ha Noi,10.090,318000,21.011667,105.847778,31516.352825


## 2. Explore using Foursquare

In [4]:
!pip install folium
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [5]:
address = 'Vietnam'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Vietnam are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Vietnam are 13.2904027, 108.4265113.


In [6]:
# create map of Vietnam using latitude and longitude values
map_vn = folium.Map(location=[latitude, longitude], zoom_start=6)

# add markers to map
for lat, lng, district, city in zip(vn['Latitude'], vn['Longitude'], vn['District'], vn['City']):
    label = '{}, {}'.format(district, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_vn)  
    
map_vn

In [7]:
CLIENT_ID = 'I00ZDGNGZBOTYZEWYOJH3GVQAICB5G1PIFXO45PKPFTYC0OM' # your Foursquare ID
CLIENT_SECRET = 'I5VCEX4X2XMFKK04OL3KI4CXVCX4O4GPAJE0LD02L3QSW5LY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: I00ZDGNGZBOTYZEWYOJH3GVQAICB5G1PIFXO45PKPFTYC0OM
CLIENT_SECRET:I5VCEX4X2XMFKK04OL3KI4CXVCX4O4GPAJE0LD02L3QSW5LY


## Get the top 100 venues

In [8]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
LIMIT = 100
radius = 2000
vn_venues = getNearbyVenues(names=vn['District'],
                                   latitudes=vn['Latitude'],
                                   longitudes=vn['Longitude']
                                  )

Ba Dinh
Bac Tu Liem
Cau Giay
Dong Da
Hai Ba Trung
Ha Dong
Hoan Kiem
Hoang Mai
Long Bien
Nam Tu Liem
Tay Ho
Thanh Xuan
Quan 1
Quan 2
Quan 3
Quan 4
Quan 5
Quan 6
Quan 7
Quan 8
Quan 9
Quan 10
Quan 11
Quan 12
Binh Thanh
Thu Duc
Go Vap
Phu Nhuan
Tan Binh
Tan Phu
Binh Tan
Cam Le
Hai Chau
Hoa Vang
Lien Chieu
Ngu Hanh Son
Son Tra
Thanh Khe


In [11]:
print(vn_venues.shape)
vn_venues.tail()

(381, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
376,Cam Le,16.016667,108.202778,Mỳ quảng Dì vân,16.018154,108.204996,Asian Restaurant
377,Cam Le,16.016667,108.202778,A4 Cafe,16.014535,108.199816,Café
378,Cam Le,16.016667,108.202778,100K Studio,16.016044,108.198364,Art Gallery
379,Hai Chau,16.060556,108.196111,Đầm Sen Quán,16.062066,108.191925,Vietnamese Restaurant
380,Lien Chieu,16.123889,108.117778,Ga Liên Chiểu,16.121328,108.120861,Train Station


In [12]:
vn_venues.groupby('District').count()

,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Ba Dinh,9,9,9,9,9,9
Bac Tu Liem,1,1,1,1,1,1
Binh Tan,1,1,1,1,1,1
Binh Thanh,14,14,14,14,14,14
Cam Le,5,5,5,5,5,5
Cau Giay,3,3,3,3,3,3
Dong Da,16,16,16,16,16,16
Go Vap,3,3,3,3,3,3
Ha Dong,4,4,4,4,4,4


In [13]:
print('There are {} uniques categories.'.format(len(vn_venues['Venue Category'].unique())))

There are 104 uniques categories.


In [14]:
## Analyze each district
# one hot encoding
vn_onehot = pd.get_dummies(vn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
vn_onehot['District'] = vn_venues['District'] 

# move neighborhood column to the first column
fixed_columns = [vn_onehot.columns[-1]] + list(vn_onehot.columns[:-1])
vn_onehot = vn_onehot[fixed_columns]

vn_onehot.head()

,District,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bar,Baseball Stadium,Basketball Court,Basketball Stadium,Beer Garden,Bistro,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Business Service,Café,Castle,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Convention Center,Cupcake Shop,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Electronics Store,Event Space,Farm,Fast Food Restaurant,Flea Market,Food,Food Court,Food Truck,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gym,Gym / Fitness Center,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Korean Restaurant,Lounge,Market,Massage Studio,Mattress Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Music Venue,Noodle House,Notary,Palace,Park,Pizza Place,Plaza,Pool,Pub,Restaurant,Rock Club,Seafood Restaurant,Shopping Mall,Smoothie Shop,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Spa,Spanish Restaurant,Speakeasy,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Temple,Thai Restaurant,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Shop,Women's Store,Zoo Exhibit
0,Ba Dinh,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ba Dinh,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Ba Dinh,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,Ba Dinh,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Ba Dinh,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
vn_onehot.shape

(381, 105)

In [16]:
vn_grouped = vn_onehot.groupby('District').mean().reset_index()
vn_grouped

,District,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bar,Baseball Stadium,Basketball Court,Basketball Stadium,Beer Garden,Bistro,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Business Service,Café,Castle,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Convention Center,Cupcake Shop,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Electronics Store,Event Space,Farm,Fast Food Restaurant,Flea Market,Food,Food Court,Food Truck,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gym,Gym / Fitness Center,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Korean Restaurant,Lounge,Market,Massage Studio,Mattress Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Music Venue,Noodle House,Notary,Palace,Park,Pizza Place,Plaza,Pool,Pub,Restaurant,Rock Club,Seafood Restaurant,Shopping Mall,Smoothie Shop,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Spa,Spanish Restaurant,Speakeasy,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Temple,Thai Restaurant,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Shop,Women's Store,Zoo Exhibit
0,Ba Dinh,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.111111,0.000000,0.000000,0.222222,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0000,0.000000,0.000000,0.111111,0.111111,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.111111,0.0000,0.0000,0.000000,0.00000,0.000000,0.0000,0.111111,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.111111,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bac Tu Liem,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.00000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
2,Binh Tan,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.00000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Binh Thanh,0.0000,0.0,

In [17]:
#Print each district along with top 5
num_top_venues = 5

for hood in vn_grouped['District']:
    print("----"+hood+"----")
    temp = vn_grouped[vn_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ba Dinh----
            venue  freq
0     Coffee Shop  0.22
1           Plaza  0.11
2          Palace  0.11
3  History Museum  0.11
4          Temple  0.11


----Bac Tu Liem----
                   venue  freq
0  Vietnamese Restaurant   1.0
1       Arepa Restaurant   0.0
2        Motorcycle Shop   0.0
3                   Pool   0.0
4                  Plaza   0.0


----Binh Tan----
              venue  freq
0  Business Service   1.0
1  Arepa Restaurant   0.0
2         Rock Club   0.0
3               Pub   0.0
4              Pool   0.0


----Binh Thanh----
                           venue  freq
0  Vegetarian / Vegan Restaurant  0.14
1                           Café  0.14
2                            Gym  0.07
3           Fast Food Restaurant  0.07
4               Asian Restaurant  0.07


----Cam Le----
              venue  freq
0              Café   0.6
1       Art Gallery   0.2
2  Asian Restaurant   0.2
3     Movie Theater   0.0
4               Pub   0.0


----Cau Giay----
          

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
district_venues_sorted = pd.DataFrame(columns=columns)
district_venues_sorted['District'] = vn_grouped['District']

for ind in np.arange(vn_grouped.shape[0]):
    district_venues_sorted.iloc[ind, 1:] = return_most_common_venues(vn_grouped.iloc[ind, :], num_top_venues)

district_venues_sorted.head()

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ba Dinh,Coffee Shop,Monument / Landmark,Palace,History Museum,Historic Site,Temple,Castle,Plaza,Zoo Exhibit,Diner
1,Bac Tu Liem,Vietnamese Restaurant,Zoo Exhibit,Food Truck,Dim Sum Restaurant,Diner,Dog Run,Electronics Store,Event Space,Farm,Fast Food Restaurant
2,Binh Tan,Business Service,Zoo Exhibit,Dessert Shop,Diner,Dog Run,Electronics Store,Event Space,Farm,Fast Food Restaurant,Flea Market
3,Binh Thanh,Vegetarian / Vegan Restaurant,Café,French Restaurant,Food,Food Truck,Fast Food Restaurant,Massage Studio,Gym,Spa,Coffee Shop
4,Cam Le,Café,Art Gallery,Asian Restaurant,Zoo Exhibit,French Restaurant,Dog Run,Electronics Store,Event Space,Farm,Fast Food Restaurant


## Run k-means to cluster the neighborhood into 5 clusters.

In [20]:
district_venues_sorted.shape

(32, 11)

In [22]:
# set number of clusters
kclusters = 4

vn_grouped_clustering = vn_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(vn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 3, 1, 0, 1, 1, 1, 0, 1], dtype=int32)

In [28]:
# add clustering labels
#district_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

vn_merged = vn

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
vn_merged = vn_merged.join(district_venues_sorted.set_index('District'), on='District')
vn_merged['Cluster Labels'] = vn_merged['Cluster Labels'].astype(int)
vn_merged.head() # check the last columns!

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [25]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(vn_merged['Latitude'], vn_merged['Longitude'], vn_merged['District'], vn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

TypeError: list indices must be integers or slices, not float

In [26]:
vn_merged.head(30)

,District,City,Area,Population,Latitude,Longitude,Density,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ba Dinh,Ha Noi,9.224,247100,21.036667,105.836111,26788.811795,1.0,Coffee Shop,Monument / Landmark,Palace,History Museum,Historic Site,Temple,Castle,Plaza,Zoo Exhibit,Diner
1,Bac Tu Liem,Ha Noi,43.350,333300,21.070351,105.762098,7688.581315,2.0,Vietnamese Restaurant,Zoo Exhibit,Food Truck,Dim Sum Restaurant,Diner,Dog Run,Electronics Store,Event Space,Farm,Fast Food Restaurant
2,Cau Giay,Ha Noi,12.040,266800,21.028307,105.790783,22159.468439,1.0,Japanese Restaurant,BBQ Joint,Park,Zoo Exhibit,Food Truck,Diner,Dog Run,Electronics Store,Event Space,Farm
3,Dong Da,Ha Noi,9.960,420900,21.013825,105.828677,42259.036145,1.0,Café,Arepa Restaurant,Park,Notary,Coffee Shop,Rock Club,Movie Theater,Soba Restaurant,Motorcycle Shop,Dessert Shop
4,Hai Ba Trung,Ha Noi,10.090,318000,21.011667,105.847778,31516.352825,1.0,Japanese Restaurant,Coffee Shop,Vietnamese Restaurant,BBQ Joint,Café,Noodle House,Korean Restaurant,Frozen Yogurt Shop,Dessert Shop,Multiplex
5,Ha Dong,Ha Noi,47.917,319800,20.966667,105.766667,6674.040528,0.0,Café,Bakery,Zoo Exhibit,French Restaurant,Diner,Dog Run,Electronics Store,Event Space,Farm,Fast Food Restaurant
6,Hoan Kiem,Ha Noi,5.290,160600,21.028889,105.8525,30359.168242,1.0,Hotel,Coffee Shop,Café,Vietnamese Restaurant,Noodle House,Spa,Ice Cream Shop,Juice Bar,Pizza Place,Park
7,Hoang Mai,Ha Noi,41.040,411500,20.968056,105.848333,10026.803119,1.0,Zoo Exhibit,Park,Women's Store,Arts & Crafts Store,French Restaurant,Dog Run,Art Gallery,Electronics Store,Event Space,Farm
8,Long Bien,Ha Noi,60.380,291900,21.0175,105.8858,4834.382246,1.0,Soccer Field,Farm,Zoo Exhibit,Cupcake Shop,Dim Sum Restaurant,Diner,Dog Run,Electronics Store,Event Space,Fast Food Restaurant
9,Nam Tu Liem,Ha Noi,32.270,236700,21.011526,105.762449,7334.986055,0.0,Café,Mattress Store,Zoo Exhibit,Food Truck,Diner,Dog Run,Electronics Store,Event Space,Farm,Fast Food Restaurant
